<a href="https://colab.research.google.com/github/Jahan08/Ambertools-CP2K-MM-QM-Biomolecular-Simulation/blob/main/hERG-web-chatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
     

--2023-04-17 15:51:06--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   223MB/s    in 0.4s    

2023-04-17 15:51:06 (223 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=p

In [5]:
import ssl
print(ssl.OPENSSL_VERSION)


OpenSSL 1.1.1f  31 Mar 2020


In [6]:
import os
os.environ['OPENSSL_VERSION'] = '1.1.1g'


In [7]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

AttributeError: ignored

In [ ]:
df_url = 'https://github.com/Jahan08/Amber-tutorial/raw/main/hERG_all.csv'
df = pd.read_csv(df_url)
df

,Name,assay_chembl_id,assay_description,Source,hERG_uM,Activity,value,Canonical_Smiles,Molecular_Weight
0,CHEMBL12713,CHEMBL841079,Inhibition of hERG currents Kv11.1,J Med Chem,0.0140,No,14.0,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,440.94084
1,CHEMBL1108,CHEMBL691014,K+ channel blocking activity in human embryoni...,J Med Chem,0.0322,No,32.2,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,379.42738
2,CHEMBL6966,CHEMBL691014,K+ channel blocking activity in human embryoni...,J Med Chem,0.1430,No,143.0,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,454.60162
3,CHEMBL1107,CHEMBL877203,K+ channel blocking activity in Chinese hamste...,J Med Chem,0.1960,No,196.0,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,500.42371
4,CHEMBL998,CHEMBL691014,K+ channel blocking activity in human embryoni...,J Med Chem,0.1730,No,173.0,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,382.88322
...,...,...,...,...,...,...,...,...,...
2319,CHEMBL5081517,CHEMBL5048865,Inhibition of hERG by patch clamp method,J Med Chem,44.0000,Yes,44000.0,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(C)(=O)=O)c...,601.55657
2320,CHEMBL5072442,CHEMBL5048865,Inhibition of hERG by patch clamp method,J Med Chem,33.0000,Yes,33000.0,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(=O)(=O)N4C...,606.62756
2321,CHEMBL5093378,CHEMBL5049389,Inhibition of hERG,J Med Chem,0.6700,No,670.0,Cc1nc(C)c([C@H](OC(C)(C)C)C(=O)O)c(N2CCC(C)(C)...,562.71462
2322,CHEMBL5070887,CHEMBL5050750,Inhibition of human ERG,ACS Med Chem Lett,0.0140,No,14.0,Cc1cnc(Nc2ccnn2C)nc1-c1cc2n(c1)C(=O)N([C@H](CO...,463.91948


# Generate ECFP6 fingerprints for each molecule in the SMILES column

In [ ]:
X = []
for smiles in df['Canonical_Smiles']:
    mol = Chem.MolFromSmiles(smiles)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048)
    X.append(fingerprint)

# Convert the list of fingerprints to a numpy array
X = np.array(X)

In [ ]:
Y = df['Activity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a deep neural network model
model = Sequential()
model.add(Dense(units=128, activation='relu', input_dim=2048))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

# Evaluate the model on the test set
_, accuracy = model.evaluate(X_test, y_test, batch_size=32, verbose=1)
print('Accuracy:', accuracy)

In [ ]:
from sklearn.metrics import roc_auc_score

# Predict probabilities for the test set
y_pred_proba = model.predict_proba(X_test)

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print('AUC Score:', auc_score)

In [ ]:
import pickle
pickle.dump(model, open('hERG_model.pkl', 'wb'))

In [ ]:
! pip3 install streamlit